Demonstrate delay correction in Scan

For this notebook, we use the "Dual 0" chain for mkids software. I

On the ZCU216 the bitfile mkids_2x2_kidsim_v2 uses these connections:

DAC output on DAC Tile = 2, DAC Ch = 0, which is the connector 0_230, on JHC3
ADC input on ADC Tile = 2, ADC Ch = 0, which is the connector 0_226, on JHC7
We use the "Sim 0" chain to simulate the resonance. It uses these connections:

DAC output on DAC Tile = 3, DAC Ch = 0, which is the connector 0_231, on JHC3
ADC input on ADC Tile = 3, ADC Ch = 0, which is the connector 0_227, on JHC7
These are connected through the low-frequency Baluns.

connector 0_230, on JHC3 <--> connector 0_227, on JHC7

connector 0_226, on JHC7 <--> connector 0_231, on JHC3

In [ ]:
import sys
sys.path.append('../../qick/qick_lib/')
sys.path.append('../soft/')
import Scan
from mkids import *
import matplotlib.pyplot as plt
import resonanceFitter
import time
import importlib
import datetime

In [ ]:
# Use this bitfile.  
bitfile = "mkids_2x2_kidsim_v2"

# Set up the firmware to use the kids and simu chains described above
iKids = 0
iSimu = 0
scan = Scan.Scan(bitfile, iKids=iKids, iSimu=iSimu)

In [ ]:
# Print out properties of the firmware
print("       Nyquist Frequency:  %.1f MHz"%scan.fNyquist)
print("      Number of channels:  nInCh=%d  nOutCh=%d"%(scan.nInCh, scan.nOutCh))
print("      Channel Separation:  %.1f MHz"%scan.dfChannel)
print("              decimation:  %d"%scan.kidsChain.analysis.decimation)
print("Sampling frequency fs_ch:  %.1f MHz"%scan.kidsChain.analysis.fs_ch)


In [ ]:
# Set the mixer to a convenient location, the center of the first Nyquist zone
fMixer = scan.fNyquist/2
scan.set_mixer(fMixer)

In [ ]:
# Place resonators

# Get the simulation chain.
simu = scan.simuChain

# Set quantization.
simu.analysis.qout(3)
simu.synthesis.qout(3)

# Disable all resonators.
simu.alloff()

fResonances = [543.21]
for fResonance in fResonances:
    print("fResonance = {} MHz".format(fResonance))
    simu.enable(fResonance)


In [ ]:
# Place one tone in the center of each bin
inChannelsAll = np.arange(scan.nInCh)
fTones = np.sort(scan.outCh2Freq(inChannelsAll))[1:-1] # Skip the lowest and highest frequencies
inChannels = scan.outFreq2ch(fTones)
plt.plot(inChannels, fTones, 'o')
plt.xlabel(" number")
plt.ylabel("center frequency (MHz)")

In [ ]:
# Set phases of the input tones
np.random.seed(123555)
phases = 2*np.pi*np.random.uniform(size=len(fTones))
gains = np.ones(len(fTones))/len(fTones)

# Do not apply compensation gain
cgs = None

bandwidth = scan.dfChannel

nf = 100

# This sweeps all of the tones simultaneously and applies the nominal delay to phase
sweptTones = scan.sweep_tones(fTones, phases, gains, cgs, bandwidth, nf)

In [ ]:

f,x = Scan.sweptTonesToSpectrum(sweptTones, fTones, scan.kidsChain.scanFOffsets)

In [ ]:
plt.plot(f,np.abs(x))
plt.xlabel("Frequency (MHz)")
plt.ylabel("Amplitude (ADUs")

In [ ]:
plt.plot(f,np.angle(x))
plt.xlabel("Frequency (MHz)")
plt.ylabel("Phase (Rad)")

In [ ]:
inds = (f>fResonances[0]-.5) & (f < fResonances[0]+.5)
plt.plot(f[inds],np.abs(x[inds]), ".-")
plt.xlabel("Frequency (MHz)")
plt.ylabel("Phase (Rad)")

In [ ]:
plt.plot(np.real(x[inds]),np.imag(x[inds]), '.-')

In [ ]:
importlib.reload(resonanceFitter)
try:
    rv = resonanceFitter.fitResonance(f[inds], x[inds])
    resonanceFitter.fitResonancePlot(f[inds], x[inds], rv[0], 0)
    resonanceFitter.fitResonancePlot(f[inds], x[inds], rv[0], 1)
    f0 = rv[0][1] # This is the fit frequency
    print("the fit worked with f0 =",f0)
except:
    f0 = fResonances[0]
    print("the fit did not work so use f0 =",f0)

In [ ]:
for v in rv[0]:
    print(v)

Set up to read phases continuously at three frequencies:  at resonance, and 10 MHz above and below

In [ ]:

fTones = np.array([f0-10, f0, f0+10])
phases = 2*np.pi*np.random.uniform(size=len(fTones))
gains = np.ones(len(fTones))/len(fTones)

# Do not apply compensation gain
cgs = None

scan.kidsChain.set_tones(fTones, phases, gains, cgs)
scan.kidsChain.enable_channels() # prepare the default, to readout all tones

In [ ]:

# This does one read to flush the buffers, since the first few samples could be stale
_ = scan.kidsChain.get_xs(mean=False)

# Now get actual data
xs = scan.kidsChain.get_xs(mean=False)


In [ ]:
# Print out the number of samples for each tone.  
# Note that we do not necessarily get the same number of samples for each tone,
# but in this example, since the tones are in nearby bins, we usually get the same.

for iTone, x in enumerate(xs):
    print(iTone, len(x))

In [ ]:
print("        fs",scan.kidsChain.analysis.fs)
print("     fc_ch",scan.kidsChain.analysis.fc_ch)
print("decimation",scan.kidsChain.analysis.decimation)
print("     fs_ch",scan.kidsChain.analysis.fs_ch)


In [ ]:
# fs_ch is the sampling frequency.  I takes into account the decimation
times = np.arange(len(xs[1]))/scan.kidsChain.analysis.fs_ch
plt.plot(times, np.angle(xs[1]),',')
plt.xlabel("Time ($\mu$sec)")
plt.ylabel("phase (Rad)")

In [ ]:
# Double the decimation, effective sampling rate is reduced, duration twice as long
# Can't go faster than decimation=2 for the ZCU216
scan.kidsChain.set_decimation(4)
# after changing a setting read one buffer
_ = scan.kidsChain.get_xs(mean=False)

times = np.arange(len(xs[1]))/scan.kidsChain.analysis.fs_ch

# Read again data
xs = scan.kidsChain.get_xs(mean=False)
plt.plot(times, np.angle(xs[1]),',')
plt.xlabel("Time ($\mu$sec)")
plt.ylabel("phase (Rad)")

In [ ]:
nRead = 100
means = np.zeros(nRead)
seconds = np.zeros(nRead)
n0 = datetime.datetime.now()
for iRead in range(nRead):
    xs = scan.kidsChain.get_xs(mean=True)
    means[iRead] = np.angle(xs[1])
    n1 = datetime.datetime.now()
    seconds[iRead] = (n1-n0).total_seconds()
seconds -= seconds[0]
plt.plot(seconds,means, '.')
plt.xlabel("time (seconds)")
plt.ylabel("phase on resonance [Rad]")

In [ ]:
import datetime
n0 = datetime.datetime.now()
time.sleep(1)
n1 = datetime.datetime.now()

In [ ]:
d = n1-n0

In [ ]:
d.total_seconds()